In [ ]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

## Getting set up

In [ ]:
comp = 'playground-series-s3e26'
path = setup_comp(comp, install='')

In [ ]:
path

Path('playground-series-s3e26')

In [ ]:
trn_path = path/'train.csv'

In [ ]:
import pandas as pd
df0 = pd.read_csv(trn_path)

In [ ]:
def process(df):
    df_ = df.copy()
    df_.Edema = df.Edema.map({'S':'Y', 'N':'N', 'Y':'Y'}) # must map all keys
    return df_

In [ ]:
df = process(df0)

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, FunctionTransformer, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold, cross_val_score, cross_validate
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.metrics import make_scorer, mean_absolute_error, classification_report, log_loss
from scipy.stats import loguniform

In [ ]:
from lightgbm import LGBMRegressor, LGBMClassifier, log_evaluation, early_stopping
from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GroupKFold
from tqdm import tqdm
from functools import partial

In [ ]:
X = df.iloc[:,1:-1]
y = df.iloc[:,-1]

In [ ]:
clf = HistGradientBoostingClassifier(max_iter=10000, early_stopping=True, max_depth=9, n_iter_no_change=50,
                                       l2_regularization=0.0026834539316593453, learning_rate=0.00806838423213153, )
ct = make_column_transformer(
            (StandardScaler(), make_column_selector(dtype_include = np.number)),
            (OneHotEncoder(drop='if_binary'), make_column_selector(dtype_include=object)), 
            remainder = 'drop')
model = make_pipeline(ct, clf)
cv_scores = cross_validate(model, X, y, cv=10, scoring='neg_log_loss', return_estimator=True)

In [ ]:
[est[-1].n_iter_ for est in cv_scores['estimator']]

[440, 365, 708, 419, 435, 429, 463, 389, 484, 388]

In [ ]:
-cv_scores['test_score'].mean()

0.4527571754876969

## Submitting to Kaggle

In [ ]:
ss = pd.read_csv(path/'sample_submission.csv')
ss

,id,Status_C,Status_CL,Status_D
0,7905,0.628084,0.034788,0.337128
1,7906,0.628084,0.034788,0.337128
2,7907,0.628084,0.034788,0.337128
3,7908,0.628084,0.034788,0.337128
4,7909,0.628084,0.034788,0.337128
...,...,...,...,...
5266,13171,0.628084,0.034788,0.337128
5267,13172,0.628084,0.034788,0.337128
5268,13173,0.628084,0.034788,0.337128
5269,13174,0.628084,0.034788,0.337128


In [ ]:
tst = pd.read_csv(path/'test.csv')
tst = process(tst)
tst.head()

,id,N_Days,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,7905,3839,D-penicillamine,19724,F,N,Y,N,N,1.2,546.0,3.37,65.0,1636.0,151.90,90.0,430.0,10.6,2.0
1,7906,2468,D-penicillamine,14975,F,N,N,N,N,1.1,660.0,4.22,94.0,1257.0,151.90,155.0,227.0,10.0,2.0
2,7907,51,Placebo,13149,F,N,Y,N,Y,2.0,151.0,2.96,46.0,961.0,69.75,101.0,213.0,13.0,4.0
3,7908,2330,D-penicillamine,20510,F,N,N,N,N,0.6,293.0,3.85,40.0,554.0,125.55,56.0,270.0,10.6,2.0
4,7909,1615,D-penicillamine,21904,F,N,Y,N,N,1.4,277.0,2.97,121.0,1110.0,125.00,126.0,221.0,9.8,1.0


In [ ]:
tst_pred = np.stack([est.predict_proba(tst.iloc[:,1:]) for est in cv_scores['estimator']]).mean(0)

In [ ]:
ss.iloc[:,1:] = tst_pred

In [ ]:
ss

,id,Status_C,Status_CL,Status_D
0,7905,0.677424,0.021248,0.301328
1,7906,0.623669,0.195182,0.181149
2,7907,0.038282,0.008089,0.953628
3,7908,0.941339,0.005477,0.053183
4,7909,0.799163,0.024871,0.175966
...,...,...,...,...
5266,13171,0.879776,0.040033,0.080191
5267,13172,0.975759,0.002274,0.021966
5268,13173,0.884534,0.005120,0.110346
5269,13174,0.985327,0.003708,0.010965


In [ ]:
ss.to_csv('subm.csv', index=False)
!head subm.csv

id,Status_C,Status_CL,Status_D
7905,0.6774237470966451,0.021248154698496537,0.3013280982048584
7906,0.6236689898393002,0.19518185710408142,0.18114915305661836
7907,0.03828227018856696,0.00808935080474088,0.9536283790066923
7908,0.9413392838879039,0.0054774430857788365,0.05318327302631719
7909,0.7991634818003521,0.02487076594166253,0.17596575225798539
7910,0.9878399375036061,0.0011399529366727416,0.011020109559721034
7911,0.9721430371938757,0.00313295207057507,0.024724010735549327
7912,0.2532543112182329,0.014004851283060882,0.7327408374987063
7913,0.009814332661048185,0.0006956630440577763,0.989490004294894


In [ ]:
# if not iskaggle:
#     from kaggle import api
#     api.competition_submit_cli('subm.csv', 'avg cv estimators', comp)

In [ ]:
!kaggle competitions submit -c playground-series-s3e26 -f subm.csv -m 'avg cv estimators'

100%|█████████████████████████████████████████| 336k/336k [00:01<00:00, 197kB/s]
Successfully submitted to Multi-Class Prediction of Cirrhosis Outcomes

## Conclusion

## Addendum

In [ ]:
if not iskaggle:
    push_notebook('xy', 'histgbr-minmax-transform',
                  title='Minmax transform and HistGBR model',
                  file='01-histgbr-minmax-transform.ipynb',
                  competition=comp, private=False, gpu=False)

Your kernel title does not resolve to the specified id. This may result in surprising behavior. We suggest making your title something that resolves to the specified id. See https://en.wikipedia.org/wiki/Clean_URL#Slug for more information on how slugs are determined.
Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/xiaochuanyang/minmax-transform-and-histgbr-model
